In [141]:
import folium
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import warnings
warnings.filterwarnings('ignore')

# Load the data

In [178]:
u00 = pd.read_csv('studentlife/dataset/sensing/gps/gps_u00.csv')
u01 = pd.read_csv('studentlife/dataset/sensing/gps/gps_u01.csv')
u02 = pd.read_csv('studentlife/dataset/sensing/gps/gps_u02.csv')
u03 = pd.read_csv('studentlife/dataset/sensing/gps/gps_u03.csv')
u04 = pd.read_csv('studentlife/dataset/sensing/gps/gps_u04.csv')
u05 = pd.read_csv('studentlife/dataset/sensing/gps/gps_u05.csv')

In [212]:
names = ['data1','data2','data3','data4','data5','data6']
files = [u00,u01,u02,u03,u04,u05]
dataset = dict(zip(names,files))

In [213]:
sum=0
for data in dataset:
    print(data + ' row counts : '+ str(len(dataset[data])))
    sum+=len(dataset[data])
print('total data: '+str(sum))

data1 row counts : 4316
data2 row counts : 5541
data3 row counts : 6607
data4 row counts : 3502
data5 row counts : 3620
data6 row counts : 5388
total data: 28974


# Reset the data

In [214]:
data = pd.concat(files)
# make index name
data.index.rename('data_ID',inplace=True)
data = data.reset_index()

In [215]:
data = data.rename(index=str, columns={"data_ID": "time", 
                               "time": "provider",
                               "provider":"network_type",
                               "network_type":"accuracy",
                               "accuracy":"latitude",
                               "latitude":"longitude",
                               "longitude":"altitude",
                               "altitude":"bearing",
                               "bearing":"speed",
                               "speed":"travelstate",
                               "travelstate":"drop"})
del data['drop']

# Map

In [183]:
locations = list(zip(data.latitude[::50], data.longitude[::50]))

In [184]:
map_img = folium.Map()

In [185]:
for i in range(len(locations)):
    folium.Marker([locations[i][0],locations[i][1]], icon=folium.Icon()).add_to(map_img)
map_img

# Clustering

### Kmeans

In [186]:
kmeans = KMeans()

In [361]:
data.dropna()
feature = data.drop(columns=['provider']).dropna()
feature.loc[feature['network_type'] == 'wifi', 'network_type'] = 0
feature.loc[feature['network_type'] == 'cell', 'network_type'] = 1
feature.loc[feature['travelstate'] == 'stationary', 'travelstate'] = 0
feature.loc[feature['travelstate'] == 'moving', 'travelstate'] = 1

In [353]:
model = KMeans(n_clusters=2)
model.fit(feature)
pred_prov = pd.DataFrame(model.predict(feature))

In [354]:
feature.reset_index(drop='index',inplace=True)

In [355]:
feature['predict_provider'] = pd.DataFrame(pred_prov, index=feature.index)

In [357]:
feature.tail()

,time,network_type,accuracy,latitude,longitude,altitude,bearing,speed,travelstate,predict_provider
27522,1369835901,0,35.355,43.709074,-72.284190,0.0,0.0,0.0,0,1
27523,1369836501,0,39.701,43.709114,-72.284157,0.0,0.0,0.0,0,1
27524,1369837101,0,38.058,43.709106,-72.284185,0.0,0.0,0.0,0,1
27525,1369837701,0,39.315,43.709117,-72.284174,0.0,0.0,0.0,0,1
27526,1369838301,0,38.680,43.709109,-72.284176,0.0,0.0,0.0,0,1


#### DBSCAN

In [358]:
dbscan = DBSCAN()

In [385]:
data.dropna()
feature = data.drop(columns=['provider']).dropna()
feature.loc[feature['network_type'] == 'wifi', 'network_type'] = 0
feature.loc[feature['network_type'] == 'cell', 'network_type'] = 1
feature.loc[feature['travelstate'] == 'stationary', 'travelstate'] = 0
feature.loc[feature['travelstate'] == 'moving', 'travelstate'] = 1

In [387]:
model = DBSCAN(eps=10, min_samples=6)
predict = pd.DataFrame(model.fit_predict(feature))

In [388]:
feature.reset_index(drop='index',inplace=True)

In [389]:
feature['predict_provider'] = pd.DataFrame(predict, index=feature.index)

In [391]:
result.tail()

,time,network_type,accuracy,latitude,longitude,altitude,bearing,speed,travelstate,predict
27522,1369835901,0,35.355,43.709074,-72.284190,0.0,0.0,0.0,0,-1
27523,1369836501,0,39.701,43.709114,-72.284157,0.0,0.0,0.0,0,-1
27524,1369837101,0,38.058,43.709106,-72.284185,0.0,0.0,0.0,0,-1
27525,1369837701,0,39.315,43.709117,-72.284174,0.0,0.0,0.0,0,-1
27526,1369838301,0,38.680,43.709109,-72.284176,0.0,0.0,0.0,0,-1
